# Excel Splitter for SharePoint - Interactive Interface

This notebook provides an easy-to-use interface for splitting Excel files by reviewer and preparing them for SharePoint upload with automatic email-based sharing.

## Prerequisites
- Python 3.9 or higher
- Required packages: pandas, openpyxl, ipywidgets
- Excel file with "Reviewer" column
- (Optional) "Email Address" column for automatic SharePoint sharing
- (Optional) Word documents and permission PDFs for distribution

## Features
- 📂 Creates application-specific folder structure
- 📊 Splits Excel by reviewer with filtered views
- 📧 Automatically maps reviewers to email addresses
- 📄 Copies related documents to each folder
- 🔐 Generates SharePoint sharing script with pre-filled emails

## Step 1: Install Required Packages
Run this cell first to ensure all dependencies are installed:

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install pandas openpyxl

## Step 2: Import Required Libraries

In [ ]:
import os
import sys
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

print("✓ Libraries imported successfully")

## Step 3: Define Helper Functions

In [ ]:
def find_column(worksheet, column_name):
    """Find column index by name"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"Cannot find '{column_name}' column! Please check column name")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """Copy selected document types to destination"""
    copied_files = []
    
    if copy_word:
        # Word documents (.docx and .doc)
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        # PDF documents
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def create_sharepoint_script(base_dir, reviewer_emails):
    """Create PowerShell script for SharePoint permissions with automatic email mapping"""
    script_path = os.path.join(base_dir, "share_folders.ps1")
    
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write("# PowerShell script to share folders on SharePoint\n")
        f.write("# Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n\n")
        f.write("# IMPORTANT: The Excel files in subfolders will sync back to the original file\n")
        f.write("# when uploaded to SharePoint with proper co-authoring enabled.\n\n")
        f.write("$siteUrl = Read-Host 'Enter SharePoint site URL'\n")
        f.write("$baseFolder = Read-Host 'Enter base folder path on SharePoint'\n\n")
        f.write("Connect-PnPOnline -Url $siteUrl -UseWebLogin\n\n")
        
        for reviewer_name, email in reviewer_emails.items():
            f.write(f"# Share folder for {reviewer_name}\n")
            f.write(f"$folderPath = Join-Path $baseFolder '{reviewer_name}'\n")
            
            if email and email != 'N/A':
                f.write(f"$userEmail = '{email}'\n")
                f.write(f"Write-Host 'Sharing with {reviewer_name} ({email})...'\n")
            else:
                f.write(f"$userEmail = Read-Host 'Enter email for {reviewer_name}'\n")
            
            f.write(f"try {{\n")
            f.write(f"    Set-PnPFolderPermission -List 'Documents' -Identity $folderPath -User $userEmail -AddRole 'Edit'\n")
            f.write(f"    Write-Host '✓ Shared folder for {reviewer_name} with Edit permissions' -ForegroundColor Green\n")
            f.write(f"}} catch {{\n")
            f.write(f"    Write-Host '✗ Failed to share with {reviewer_name}: $_' -ForegroundColor Red\n")
            f.write(f"}}\n\n")
    
    return script_path

print("✓ Helper functions defined")

## Step 4: User Input Interface
Fill in the required information below:

In [ ]:
# Create input widgets
excel_file = widgets.Text(
    value='',
    placeholder='C:\\Users\\YourName\\Documents\\user_listing.xlsx',
    description='Excel File:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

reviewer_column = widgets.Text(
    value='Reviewer',
    description='Reviewer Column:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

email_column = widgets.Text(
    value='Email Address',
    description='Email Column:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

copy_word_docs = widgets.Checkbox(
    value=True,
    description='Copy Word documents (.doc, .docx) to reviewer folders',
    style={'description_width': 'initial'}
)

copy_pdf_docs = widgets.Checkbox(
    value=True,
    description='Copy PDF documents (.pdf) to reviewer folders',
    style={'description_width': 'initial'}
)

create_script = widgets.Checkbox(
    value=True,
    description='Create SharePoint sharing script',
    style={'description_width': 'initial'}
)

auto_email = widgets.Checkbox(
    value=True,
    description='Use email addresses from Excel for automatic sharing (if available)',
    style={'description_width': 'initial'}
)

# Display input form
display(HTML("<h3>Configuration</h3>"))
display(excel_file)
display(reviewer_column)
display(email_column)
display(HTML("<h4>Options</h4>"))
display(copy_word_docs)
display(copy_pdf_docs)
display(create_script)
display(auto_email)

# Create button
process_button = widgets.Button(
    description='Process Excel File',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

output = widgets.Output()

display(HTML("<br>"))
display(process_button)
display(output)

## Step 5: Processing Function

In [ ]:
def process_excel(button):
    """Main processing function triggered by button click"""
    with output:
        clear_output()
        
        # Validate inputs
        if not excel_file.value:
            print("❌ Error: Please specify an Excel file path")
            return
        
        file_path = excel_file.value.strip()
        column = reviewer_column.value.strip()
        email_col = email_column.value.strip() if auto_email.value else None
        
        # Convert path for Windows
        file_path = os.path.normpath(file_path)
        
        if not os.path.exists(file_path):
            print(f"❌ Error: File not found: {file_path}")
            return
        
        print(f"📁 Processing: {file_path}")
        print(f"📊 Reviewer Column: {column}")
        if email_col:
            print(f"📧 Email Column: {email_col}")
        print("="*50)
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column not in df.columns:
                print(f"❌ Error: Column '{column}' not found in Excel file")
                print(f"Available columns: {', '.join(df.columns)}")
                return
            
            # Get unique reviewers
            reviewers = df[column].dropna().unique().tolist()
            print(f"✓ Found {len(reviewers)} reviewers")
            
            # Create email mapping if email column exists
            reviewer_emails = {}
            if email_col and email_col in df.columns:
                print(f"✓ Found '{email_col}' column - will use for automatic sharing")
                for reviewer in reviewers:
                    reviewer_data = df[df[column] == reviewer]
                    if not reviewer_data.empty:
                        email = reviewer_data[email_col].iloc[0]
                        if pd.notna(email):
                            reviewer_emails[str(reviewer).strip()] = str(email).strip()
                        else:
                            reviewer_emails[str(reviewer).strip()] = 'N/A'
            else:
                if email_col and email_col not in df.columns:
                    print(f"⚠️ Warning: Email column '{email_col}' not found - will prompt for emails during sharing")
                reviewer_emails = {str(r).strip(): 'N/A' for r in reviewers}
            
            # Get base directory and filename parts
            base_dir = os.path.dirname(file_path)
            base_name = os.path.basename(file_path)
            name_without_ext = os.path.splitext(base_name)[0]
            ext = os.path.splitext(base_name)[1]
            
            # Count documents in the folder
            if copy_word_docs.value or copy_pdf_docs.value:
                doc_patterns = []
                if copy_word_docs.value:
                    doc_patterns.extend(["*.docx", "*.doc"])
                if copy_pdf_docs.value:
                    doc_patterns.append("*.pdf")
                
                doc_count = 0
                for pattern in doc_patterns:
                    doc_count += len(glob.glob(os.path.join(base_dir, pattern)))
                if doc_count > 0:
                    print(f"✓ Found {doc_count} document(s) to copy")
            
            # Process each reviewer
            processed = 0
            
            for reviewer in reviewers:
                reviewer_name = str(reviewer).strip()
                print(f"\n📝 Processing: {reviewer_name}")
                
                # Create reviewer folder in the same directory as Excel file
                reviewer_folder = os.path.join(base_dir, reviewer_name)
                os.makedirs(reviewer_folder, exist_ok=True)
                
                # Create filtered Excel with reviewer name in filename
                new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
                dst_path = os.path.join(reviewer_folder, new_filename)
                wb = load_workbook(file_path)
                ws = wb.active
                
                try:
                    # Find column and apply filter
                    col_idx = find_column(ws, column)
                    max_row = ws.max_row
                    max_col = ws.max_column
                    
                    filter_range = f"A1:{get_column_letter(max_col)}{max_row}"
                    ws.auto_filter.ref = filter_range
                    ws.auto_filter.add_filter_column(col_idx - 1, [reviewer_name])
                    
                    # Hide rows that don't belong to this reviewer
                    for row in range(2, max_row + 1):  # Start from row 2 (skip header)
                        cell_value = ws.cell(row=row, column=col_idx).value
                        if cell_value != reviewer_name:
                            ws.row_dimensions[row].hidden = True
                    
                    wb.save(dst_path)
                    print(f"  ✓ Created: {new_filename}")
                    
                    # Copy selected documents
                    if copy_word_docs.value or copy_pdf_docs.value:
                        copied = copy_selected_documents(base_dir, reviewer_folder, 
                                                       copy_word=copy_word_docs.value, 
                                                       copy_pdf=copy_pdf_docs.value)
                        if copied:
                            doc_types = []
                            if copy_word_docs.value:
                                doc_types.append("Word")
                            if copy_pdf_docs.value:
                                doc_types.append("PDF")
                            print(f"  ✓ Copied {len(copied)} {'/'.join(doc_types)} document(s)")
                    
                    processed += 1
                    
                except Exception as e:
                    print(f"  ❌ Error: {e}")
                finally:
                    wb.close()
            
            # Create SharePoint script if requested
            if create_script.value:
                script_path = create_sharepoint_script(base_dir, reviewer_emails)
                print(f"\n✓ Created SharePoint script: {os.path.basename(script_path)}")
                
                # Display email mapping summary
                if any(email != 'N/A' for email in reviewer_emails.values()):
                    print("\n📧 Email addresses found:")
                    for reviewer, email in reviewer_emails.items():
                        if email != 'N/A':
                            print(f"  • {reviewer}: {email}")
                        else:
                            print(f"  • {reviewer}: [No email - will be prompted]")
            
            # Summary
            print("\n" + "="*50)
            print("✅ PROCESSING COMPLETE!")
            print(f"📊 Processed {processed}/{len(reviewers)} reviewers")
            print(f"📁 Output location: {base_dir}")
            print(f"📂 Created folders: {', '.join([str(r) for r in reviewers[:5]])}", end='')
            if len(reviewers) > 5:
                print(f" and {len(reviewers)-5} more...")
            else:
                print()
            
            print("\n⚠️  IMPORTANT: SharePoint Co-authoring")
            print("When uploaded to SharePoint, changes in the reviewer Excel files")
            print("will automatically sync back to the original file if co-authoring is enabled.")
            
            print("\n📋 Next steps:")
            print("1. Review the created folders and files")
            print("2. Upload the entire folder structure to SharePoint")
            if create_script.value:
                print("3. Run 'share_folders.ps1' to set permissions")
                if any(email != 'N/A' for email in reviewer_emails.values()):
                    print("   (Email addresses have been pre-filled where available)")
            
        except Exception as e:
            print(f"\n❌ Fatal error: {e}")
            import traceback
            traceback.print_exc()

# Attach the function to button
process_button.on_click(process_excel)
print("✓ Processing function ready")

## Step 6: Preview Excel File (Optional)
Run this cell to preview your Excel file and verify the column names:

In [ ]:
# Preview Excel file
preview_file = widgets.Text(
    value='',
    placeholder='C:\\Users\\YourName\\Documents\\user_listing.xlsx',
    description='File to Preview:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

preview_button = widgets.Button(
    description='Preview Excel',
    button_style='info'
)

preview_output = widgets.Output()

def preview_excel(button):
    with preview_output:
        clear_output()
        file_path = preview_file.value.strip()
        if not file_path:
            print("Please enter a file path")
            return
        
        file_path = os.path.normpath(file_path)
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return
        
        try:
            df = pd.read_excel(file_path, engine='openpyxl')
            print(f"📊 File: {os.path.basename(file_path)}")
            print(f"📏 Shape: {df.shape[0]} rows × {df.shape[1]} columns")
            print(f"\n📋 Columns: {', '.join(df.columns)}")
            print("\n🔍 First 5 rows:")
            display(df.head())
            
            if 'Reviewer' in df.columns:
                reviewers = df['Reviewer'].dropna().unique()
                print(f"\n👥 Unique Reviewers ({len(reviewers)}): {', '.join(str(r) for r in reviewers[:10])}")
                if len(reviewers) > 10:
                    print("... and more")
            
            if 'Email Address' in df.columns:
                print("\n📧 Email Address column found!")
                # Show sample email mapping
                sample_data = df[['Reviewer', 'Email Address']].drop_duplicates('Reviewer').head(5)
                print("\nSample reviewer-email mapping:")
                for _, row in sample_data.iterrows():
                    if pd.notna(row['Reviewer']) and pd.notna(row['Email Address']):
                        print(f"  • {row['Reviewer']}: {row['Email Address']}")
            
        except Exception as e:
            print(f"Error reading file: {e}")

preview_button.on_click(preview_excel)

display(preview_file)
display(preview_button)
display(preview_output)

## Tips for Windows Users

### File Paths
- Use full paths: `C:\Users\YourName\Documents\file.xlsx`
- Or use forward slashes: `C:/Users/YourName/Documents/file.xlsx`
- For network drives: `\\\\server\share\folder\file.xlsx`

### Excel File Naming
- Original: `user_listing.xlsx`
- Reviewer files: `user_listing - John Doe.xlsx`, `user_listing - Jane Smith.xlsx`
- Each reviewer's name is appended to the filename for easy identification

### SharePoint Co-authoring
- **IMPORTANT**: When you upload to SharePoint with co-authoring enabled, changes made in reviewer Excel files will automatically sync back to the original file
- This requires proper SharePoint setup with co-authoring features
- Each reviewer can only see and edit their filtered data, but changes sync to the master

### Document Handling
- **Word Documents**: Checkbox to copy all `.doc` and `.docx` files
- **PDF Documents**: Separate checkbox to copy all `.pdf` files
- Select which document types you want to distribute to reviewers

### Folder Structure Example
```
Original:
C:\Documents\
├── user_listing.xlsx
├── Guide.docx
└── Form.pdf

After Processing:
C:\Documents\
├── user_listing.xlsx (original)
├── Guide.docx
├── Form.pdf
├── John Doe\
│   ├── user_listing - John Doe.xlsx
│   ├── Guide.docx (if Word checkbox selected)
│   └── Form.pdf (if PDF checkbox selected)
└── Jane Smith\
    └── user_listing - Jane Smith.xlsx (etc.)
```

### Troubleshooting
- **"File not found"**: Check the path and ensure the file exists
- **"Column not found"**: Use the preview function to verify column names
- **Permission errors**: Run Jupyter as administrator if needed